In [ ]:
from indy import anoncreds, did, ledger, pool, wallet, IndyError
import json

async def steward_registration(steward):
    await create_wallet(steward)
    print("\"Sovrin Steward\" -> Create and store in Wallet DID from seed")
    steward['did_info'] = json.dumps({'seed': steward['seed']})
    steward['did'], steward['key'] = await did.create_and_store_my_did(steward['wallet'], steward['did_info'])

async def create_wallet(identity):
    print("\"{}\" -> Create wallet".format(identity['name']))
    try:
        await wallet.create_wallet(identity['wallet_config'], identity['wallet_credentials'])
    except IndyError as ex:
        if ex.error_code == 306:
            pass
    identity['wallet'] = await wallet.open_wallet(identity['wallet_config'], identity['wallet_credentials'])

async def trustAnchor_registration(from_, to):
     await getting_verinym(from_, to)

async def getting_verinym(from_, to):
    await create_wallet(to)

    (to['did'], to['key']) = await did.create_and_store_my_did(to['wallet'], "{}")
    print("In getting verinym")
    from_['info'] = {
        'did': to['did'],
        'verkey': to['key'],
        'role': to['role'] or None
    }

    await send_nym(from_['pool'], from_['wallet'], from_['did'], from_['info']['did'],
                   from_['info']['verkey'], from_['info']['role'])

async def send_nym(pool_handle, wallet_handle, _did, new_did, new_key, role):
    print("In getting verinym1")
    nym_request = await ledger.build_nym_request(_did, new_did, new_key, None, role)
    print("In getting verinym2")
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, nym_request)
    print("In getting verinym3")

async def supplier_cred_schema(supplier,supplier_certificate):
    (supplier['supplier_certificate_schema_id'], supplier['supplier_certificate_schema']) = \
        await anoncreds.issuer_create_schema(supplier['did'], supplier_certificate['name'], supplier_certificate['version'],
                                             json.dumps(supplier_certificate['attributes']))
    supplier_certificate_schema_id = supplier['supplier_certificate_schema_id']

    print("\"Supplier\" -> Send \"Supplier-Certificate\" Schema to Ledger")
    await send_schema(supplier['pool'], supplier['wallet'], supplier['did'], supplier['supplier_certificate_schema'])

async def send_schema(pool_handle, wallet_handle, _did, schema):
    schema_request = await ledger.build_schema_request(_did, schema)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, schema_request)

async def supplier_cred_definition(supplier, supplier_certificate_cred_def):

    print("\"Supplier\" -> Get from Ledger \"Supplier-Certificate\" Schema")
    supplier_certificate_schema_id = supplier['supplier_certificate_schema_id']

    (supplier['supplier_certificate_schema_id'], supplier['supplier_certificate_schema']) = \
        await get_schema(supplier['pool'], supplier['did'], supplier_certificate_schema_id)

    (supplier['supplier_certificate_cred_def_id'], supplier['supplier_certificate_cred_def']) = \
        await anoncreds.issuer_create_and_store_credential_def(supplier['wallet'], supplier['did'],
                                                               supplier['supplier_certificate_schema'],
                                                               supplier_certificate_cred_def['tag'],
                                                               supplier_certificate_cred_def['type'],
                                                               json.dumps(supplier_certificate_cred_def['config']))
    print("\"Supplier\" -> Send \"Supplier Supplier-Certificate\" Credential Definition to Ledger")
    await send_cred_def(supplier['pool'], supplier['wallet'], supplier['did'], supplier['supplier_certificate_cred_def'])

async def get_schema(pool_handle, _did, schema_id):
    get_schema_request = await ledger.build_get_schema_request(_did, schema_id)
    get_schema_response = await ledger.submit_request(pool_handle, get_schema_request)
    return await ledger.parse_get_schema_response(get_schema_response)

async def send_cred_def(pool_handle, wallet_handle, _did, cred_def_json):
    cred_def_request = await ledger.build_cred_def_request(_did, cred_def_json)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, cred_def_request)

async def device1_registration(device1):
    await create_wallet(device1)
    (device1['did'], device1['key']) = await did.create_and_store_my_did(device1['wallet'], "{}")

async def supplier_create_credOffer(supplier):
     supplier['supplier_certificate_cred_offer'] = \
        await anoncreds.issuer_create_credential_offer(supplier['wallet'], supplier['supplier_certificate_cred_def_id'])

async def get_cred_def(pool_handle, _did, cred_def_id):
    get_cred_def_request = await ledger.build_get_cred_def_request(_did, cred_def_id)
    get_cred_def_response = await ledger.submit_request(pool_handle, get_cred_def_request)
    return await ledger.parse_get_cred_def_response(get_cred_def_response)

async def retailerCreateProofRequest(retailer,supplier):
     nonce = await anoncreds.generate_nonce()
     retailer['verification_proof_request'] = json.dumps({
        'nonce': nonce,
        'name': 'Verification-Request',
        'version': '0.1',
        'requested_attributes': {
            'attr1_referent': {
                'name': 'first_name'
            },
            'attr2_referent': {
                'name': 'last_name'
            },
            'attr3_referent': {
                'name': 'degree',
                'restrictions': [{'cred_def_id': supplier['supplier_certificate_cred_def_id']}]
            },
            'attr4_referent': {
                'name': 'status',
                'restrictions': [{'cred_def_id': supplier['supplier_certificate_cred_def_id']}]
            },
            'attr5_referent': {
                'name': 'ssn',
                'restrictions': [{'cred_def_id': supplier['supplier_certificate_cred_def_id']}]
            },
            'attr6_referent': {
                'name': 'phone_number'
            },
            # 'attr7_referent': {
            #     'name': 'degree2',
            #     'restrictions': [{'cred_def_id': supplier['supplier_certificate_cred_def_id']}]
            # },
            # 'attr8_referent': {
            #     'name': 'degree3',
            #     'restrictions': [{'cred_def_id': supplier['supplier_certificate_cred_def_id']}]
            # },
            # 'attr9_referent': {
            #     'name': 'degree4',
            #     'restrictions': [{'cred_def_id': supplier['supplier_certificate_cred_def_id']}]
            # },
            # 'attr10_referent': {
            #     'name': 'middle_name'
            # },
            # 'attr11_referent': {
            #     'name': 'status2',
            #     'restrictions': [{'cred_def_id': supplier['supplier_certificate_cred_def_id']}]
            # },
            # 'attr12_referent': {
            #     'name': 'status3',
            #     'restrictions': [{'cred_def_id': supplier['supplier_certificate_cred_def_id']}]
            # },
            # 'attr13_referent': {
            #     'name': 'status4',
            #     'restrictions': [{'cred_def_id': supplier['supplier_certificate_cred_def_id']}]
            # }


         },
        'requested_predicates': {
            'predicate1_referent': {
                'name': 'average',
                'p_type': '>=',
                'p_value': 4,
                'restrictions': [{'cred_def_id': supplier['supplier_certificate_cred_def_id']}]
            }
        }
    })

async def supplierCreateCredential(supplier):
    supplier['supplier_certificate_cred'], _, _ = \
        await anoncreds.issuer_create_credential(supplier['wallet'], supplier['supplier_certificate_cred_offer'],
                                                 supplier['supplier_certificate_cred_request'],
                                                 supplier['device_supplier_certificate_cred_values'], None, None)

async def deviceCreateCredRequest(device1):
     (device1['supplier_certificate_cred_request'], device1['supplier_certificate_cred_request_metadata']) = \
        await anoncreds.prover_create_credential_req(device1['wallet'], device1['did'],
                                                     device1['supplier_certificate_cred_offer'], device1['supplier_cred_def'],
                                                     device1['master_secret_id'])

async def get_credential_for_referent(search_handle, referent):
    credentials = json.loads(
        await anoncreds.prover_fetch_credentials_for_proof_req(search_handle, referent, 10))
    return credentials[0]['cred_info']  

async def prover_get_entities_from_ledger(pool_handle, _did, identifiers, actor):
    schemas = {}
    cred_defs = {}
    rev_states = {}
    for item in identifiers.values():
        print("\"{}\" -> Get Schema from Ledger".format(actor))
        (received_schema_id, received_schema) = await get_schema(pool_handle, _did, item['schema_id'])
        schemas[received_schema_id] = json.loads(received_schema)

        print("\"{}\" -> Get Credential Definition from Ledger".format(actor))
        (received_cred_def_id, received_cred_def) = await get_cred_def(pool_handle, _did, item['cred_def_id'])
        cred_defs[received_cred_def_id] = json.loads(received_cred_def)

        if 'rev_reg_seq_no' in item:
            pass  # TODO Create Revocation States

    return json.dumps(schemas), json.dumps(cred_defs), json.dumps(rev_states)

async def verifier_get_entities_from_ledger(pool_handle, _did, identifiers, actor):
    schemas = {}
    cred_defs = {}
    rev_reg_defs = {}
    rev_regs = {}
    for item in identifiers:
        print("\"{}\" -> Get Schema from Ledger".format(actor))
        (received_schema_id, received_schema) = await get_schema(pool_handle, _did, item['schema_id'])
        schemas[received_schema_id] = json.loads(received_schema)

        print("\"{}\" -> Get Credential Definition from Ledger".format(actor))
        (received_cred_def_id, received_cred_def) = await get_cred_def(pool_handle, _did, item['cred_def_id'])
        cred_defs[received_cred_def_id] = json.loads(received_cred_def)

        if 'rev_reg_seq_no' in item:
            pass  # TODO Get Revocation Definitions and Revocation Registries

    return json.dumps(schemas), json.dumps(cred_defs), json.dumps(rev_reg_defs), json.dumps(rev_regs)


